<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/EqBalancing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
import re
import numpy as np
import pandas as pd

In [183]:
rxn="aKNO3 + 5C12H22O11 -> bN2 + cCO2 + dH2O + eK2CO3"

In [184]:
rxn=re.sub(r'\s+', '', rxn)
lhs,rhs=rxn.split('->')

In [185]:
def process(side, sign):
    stoic_pat = re.compile(r'(.*?)([A-Z].*)')
    atom_pat = re.compile(r'([A-Z][a-z]?)(\d*)')
    stoic_molecules=side.split('+')
    mol_dict={}
    for stoic_molecule in stoic_molecules:
        stoic,molecule=stoic_pat.match(stoic_molecule).group(1,2)
        stoic = sign*1 if stoic=='' else int(stoic)*sign if stoic.isnumeric() else stoic
        d={'stoic':stoic}
        for atom, num in atom_pat.findall(molecule):
            d[atom]=1 if num=='' else int(num)
        
        mol_dict[molecule]=d
    return pd.DataFrame(mol_dict)



In [186]:
df = pd.concat([process(lhs,-1),process(rhs,1)],axis=1)
df = df.fillna(0)

In [187]:
df

,KNO3,C12H22O11,N2,CO2,H2O,K2CO3
stoic,a,-5.0,b,c,d,e
K,1,0.0,0,0,0,2
N,1,0.0,2,0,0,0
O,3,11.0,0,2,1,3
C,0,12.0,0,1,0,1
H,0,22.0,0,0,2,0


In [188]:
fixed = pd.to_numeric(df.loc['stoic'],errors='coerce').notna()
atom_num = df.iloc[1:].astype(int)

In [189]:
rs=-np.sum((atom_num.loc[:,fixed]*df.loc['stoic',fixed].astype(int)),axis=1)

In [190]:
res = np.linalg.solve(atom_num.loc[:,~fixed],rs)

In [191]:
df.loc['stoic',~fixed]=res

In [192]:
df

,KNO3,C12H22O11,N2,CO2,H2O,K2CO3
stoic,-48.0,-5.0,24.0,36.0,55.0,24.0
K,1,0.0,0,0,0,2
N,1,0.0,2,0,0,0
O,3,11.0,0,2,1,3
C,0,12.0,0,1,0,1
H,0,22.0,0,0,2,0


In [193]:
lhs_react=[]
rhs_react=[]
for num, molecule in zip(df.loc['stoic'], df.columns):
    num = int(num) if abs(int(num)-num)<1e-10 else num
    if num<0:
        lhs_react.append(f'{abs(num)}{molecule}')
    else:
        rhs_react.append(f'{abs(num)}{molecule}')

In [194]:
' + '.join(lhs_react) + ' -> ' + ' + '.join(rhs_react)

'48KNO3 + 5C12H22O11 -> 24N2 + 36CO2 + 55H2O + 24K2CO3'